In [1]:
from pipeline.preprocessing import one_hot_encoding, data_wrangling, \
    feature_mfcc, feature_chromagram, feature_melspectrogram, feature_bandpower_struct
    
from pipeline.dataloader import PhonocardiogramAudioDataset, PhonocardiogramByIDDatasetOnlyResult, PhonocardiogramAugmentationTSV

from tqdm import tqdm
from pipeline.utils import compose_feature_label, audio_random_windowing

from pathlib import Path
from torch.utils.data import DataLoader
import torch
import re
import pandas as pd
import os
import numpy as np

In [2]:
file = Path(".") / "assets" / "the-circor-digiscope-phonocardiogram-dataset-1.0.3"
# Training On CSV data
original_data = pd.read_csv(str(file  / "training_data.csv"))
    
model_df = data_wrangling(original_data)
X_CSV = one_hot_encoding(model_df, [
    'Murmur', 
    'Systolic murmur quality', 
    'Systolic murmur pitch',
    'Systolic murmur grading', 
    'Systolic murmur shape', 
    'Systolic murmur timing',
    'Diastolic murmur quality', 
    'Diastolic murmur pitch',
    'Diastolic murmur grading', 
    'Diastolic murmur shape', 
    'Diastolic murmur timing',
])
y_CSV = model_df['Outcome']



# Training on actual patient audio files
segmentation_table = PhonocardiogramAugmentationTSV(file / "training_data")

def augmentation(data, sr=4000, window_length_hz=200, window_len_sec =5.):
    # This augmentation WILL conflict with new feature of frequency based extraction. ->
    x = data
    # x = energy_band_augmentation_random_win(x, sr=sr, window_hz_length=window_length_hz)
    # x = np.fft.ifft(x).real
        
    x = audio_random_windowing(x, window_len_sec)
    return x



def feature_csv(file):
    match = re.match(r'(\d+)_(AV|TV|MV|PV|Phc)', os.path.basename(file))
    key = int(match.group(1))
    record = X_CSV.loc[original_data["Patient ID"] == key].to_numpy()[0]
    return record

def compose_with_csv(file, audio_extracted_features_label):
    feature, y = audio_extracted_features_label
    csv_feat = feature_csv(file)
    return np.concatenate([feature, csv_feat], axis=0), y


features_fn = [
    feature_mfcc,
    feature_chromagram, 
    feature_melspectrogram,
    feature_bandpower_struct(4000,200,0.7),
]

c:\Python Work\Senior Design Local\HeartDiseasePrediction\pipeline\preprocessing.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Age"].fillna(
c:\Python Work\Senior Design Local\HeartDiseasePrediction\pipeline\preprocessing.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["Outcom

In [3]:
X = []
y = []

for feature in features_fn:
    lookup = PhonocardiogramByIDDatasetOnlyResult(str(file / "training_data.csv"))
    dset = PhonocardiogramAudioDataset(
        file / "training_data",
        ".wav",
        "*", # Everything
        transform=lambda f : compose_with_csv(f, compose_feature_label(
            f,
            lookup, 
            [feature],
            lambda ary_data : augmentation(ary_data,4000,300,3.))
        ),  
        balancing=True,
        csvfile=str(file / "training_data.csv"),
        shuffle=True
    )

    loader = DataLoader(
        dset, 
        batch_size=1,
        shuffle=True
        # collate_fn=lambda x : x,
    )

    X1 = []
    y1 = []

    for resample in range(BATCHING := 1):
        for i in tqdm(loader): # very slow 
            X_i,y_i = i
            X1.append(X_i)
            y1.append(y_i)

    # Creating 1 large matrix to train with classical models
    X1 = torch.cat(X1, dim=0)
    y1 = torch.cat(y1, dim=0)

    X.append(X1)
    y.append(y1)

y = y[0]


    

  0%|          | 0/3060 [00:00<?, ?it/s]C:\Users\songy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
100%|██████████| 3060/3060 [00:13<00:00, 226.28it/s]


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,Lambda, AveragePooling1D, MaxPooling1D, Flatten,Reshape, Dense, Dropout, BatchNormalization, concatenate
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
def get_cnn_with_concat(input_shapes):
    print(input_shapes)

    cnn_list = []
    input_list = []
    max_seq_length = max(input_shapes) 

    for i, input_shape in enumerate(input_shapes):
        input = tf.keras.Input(shape=(input_shape,1))
        cnn = Conv1D(filters=64, kernel_size=3, activation='relu')(input)
        cnn = Reshape((-1,64))(cnn) 
        
        padding_shape = tf.constant([[0, 0], [0, max_seq_length - input_shape], [0, 0]])
        cnn = Lambda(lambda x, padding_shape=padding_shape: tf.pad(x, padding_shape, 'CONSTANT'))(cnn)
        cnn_list.append(cnn)
        input_list.append(input)

    combined_features = concatenate(cnn_list, axis=-1)

    x = Conv1D(filters=64, kernel_size=3, activation='relu')(combined_features)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.2)(x)

    x = Conv1D(filters=128, kernel_size=3, activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.2)(x)

    # x = Conv1D(filters=256, kernel_size=3, activation='relu')(x)
    # x = BatchNormalization()(x)
    # x = MaxPooling1D(pool_size=2)(x)
    # x = Dropout(0.3)(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=input_list, outputs=output)
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model 

In [5]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score
def cnn_with_concat_train(X,y):
    y_train, y_temp = train_test_split(y, test_size=0.4, random_state=42)
    y_val, y_test = train_test_split(y_temp, test_size=0.5, random_state=42)
    X_train = []
    X_test = []
    X_val = []
    for x in X:
        x_train, x_temp = train_test_split(x, test_size=0.4, random_state=42)
        x_val, x_test = train_test_split(x_temp, test_size=0.5, random_state=42)
        X_train.append(x_train)
        X_val.append(x_val)
        X_test.append(x_test)

    cnn = get_cnn_with_concat([x_train.shape[1] for x_train in X_train])
    cnn.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), verbose=1)

    probabilities = cnn.predict(X_test)
    threshold = 0.5
    y_pred = (probabilities >= threshold).astype(int)


    acc = metrics.accuracy_score(y_test, y_pred)
    fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {acc}")
    print(f"Auc: {auc}")
    print(f"F1 Score: {f1}")
    acc = round(acc * 100, 2)
    auc = round(auc * 100, 2)
    f1 = round(f1 * 100, 2)
    return acc, auc, f1

In [6]:
acc, auc, f1 = cnn_with_concat_train(X,y) 

[72, 42, 46, 61]

Epoch 1/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5205 - loss: 1.1398 - val_accuracy: 0.5049 - val_loss: 0.6929
Epoch 2/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6363 - loss: 0.6508 - val_accuracy: 0.5163 - val_loss: 0.6910
Epoch 3/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6089 - loss: 0.6444 - val_accuracy: 0.5245 - val_loss: 0.6859
Epoch 4/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6306 - loss: 0.6261 - val_accuracy: 0.5833 - val_loss: 0.6864
Epoch 5/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6286 - loss: 0.6173 - val_accuracy: 0.6013 - val_loss: 0.6724
Epoch 6/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6379 - loss: 0.6126 - val_accuracy: 0.5163 - val_loss: 0.6709
Epoch 7/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6310 - loss: 0.6296 - val_accuracy: 0.5637 - val_loss: 0.6693
Epoch 8/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6290 - loss: 0.6363 - val_ac